In [8]:
import pandas as pd

In [9]:
def string_to_dataframe(data_string):
    # Split data into rows and columns based on delimiters
    rows = data_string.split(" <0x0A> ")
    data = [row.split(" | ") for row in rows]
    
    header = data[0] # First entry is the header
    records = data[1:] # Remaining entries are the data
    
    # Create DataFrame
    df = pd.DataFrame(records, columns=header).apply(pd.to_numeric, errors='ignore') #if data can be converted to numbers, its converted
    
    return df

In [11]:
#change to load in current csvs
data_string = "Entity | Tend to favor side one | fairly Deal with all sides <0x0A> Dem/Lean Dem | 69 | 29 <0x0A> Rep/Lean Rep | 91 | 8 <0x0A> U.S adults | 79 | 20"
df = string_to_dataframe(data_string)
display(df) #change to overwrite csvs

,Entity,Tend to favor side one,fairly Deal with all sides
0,Dem/Lean Dem,69,29
1,Rep/Lean Rep,91,8
2,U.S adults,79,20
